In [1]:
# !pip install fuzzywuzzy

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from collections import Counter
from fuzzywuzzy import process, fuzz
import pickle

/usr/local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Cargamos los datos

In [3]:
# cargamos el csv donde tenemos todos los títulos que tenemos en Netflix
df_titulos = pd.read_csv("../data/netflix_titles.csv", index_col = 0 )
df_titulos.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


In [4]:
# cargamos el csv donde tenemos todas las producciones de Netflix
df_original = pd.read_csv("../data/netflix_originals.csv", index_col = 0)
df_original.head(2)

,Title,Genre,Premiere,Runtime,IMDB Score,Language
0,Enter the Anime,Documentary,"August 5, 2019",58,2.5,English/Japanese
1,Dark Forces,Thriller,"August 21, 2020",81,2.6,Spanish


In [5]:
df_titulos.shape

(8807, 12)

In [6]:
df_original.shape

(513, 6)

## Juntamos los dos dataframes

In [7]:
# juntamos los dos dataframes
df = df_titulos.merge(df_original, left_on= "title", right_on="Title", how="inner")
df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Title,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Dick Johnson Is Dead,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada","September 1, 2021",2015,R,82 min,Action & Adventure,"When a retired CIA agent is kidnapped, his son...",Extraction,Action,"April 24, 2020",117,6.7,English


In [8]:

# Dropping the columns that are not needed for the analysis.
df.drop(['release_year', 'Runtime', 'description','Title', "show_id"], axis = 1, inplace=True)
df.head(2)

,type,title,director,cast,country,date_added,rating,duration,listed_in,Genre,Premiere,IMDB Score,Language
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",PG-13,90 min,Documentaries,Documentary,"October 2, 2020",7.5,English
1,Movie,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada","September 1, 2021",R,82 min,Action & Adventure,Action,"April 24, 2020",6.7,English


Tenemos columnas en mayúsculas y otras en minúsculas, y con espacios. Vamos a intentar homogeneizarlo:

In [9]:
columnas_nuevas = {col : col.replace(" ", "_").lower() for col in df.columns}
print(columnas_nuevas)

{'type': 'type', 'title': 'title', 'director': 'director', 'cast': 'cast', 'country': 'country', 'date_added': 'date_added', 'rating': 'rating', 'duration': 'duration', 'listed_in': 'listed_in', 'Genre': 'genre', 'Premiere': 'premiere', 'IMDB Score': 'imdb_score', 'Language': 'language'}


In [10]:
df.rename(columns = columnas_nuevas, inplace = True)
df.head(2)

,type,title,director,cast,country,date_added,rating,duration,listed_in,genre,premiere,imdb_score,language
0,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",PG-13,90 min,Documentaries,Documentary,"October 2, 2020",7.5,English
1,Movie,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada","September 1, 2021",R,82 min,Action & Adventure,Action,"April 24, 2020",6.7,English


# Análisis exploratorio

In [11]:
# Número de filas  y columnas del dataframe
df.shape

(513, 13)

In [12]:
# Información básica de cada una de las columnas del df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 0 to 512
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   type        513 non-null    object 
 1   title       513 non-null    object 
 2   director    491 non-null    object 
 3   cast        422 non-null    object 
 4   country     499 non-null    object 
 5   date_added  513 non-null    object 
 6   rating      513 non-null    object 
 7   duration    276 non-null    object 
 8   listed_in   513 non-null    object 
 9   genre       513 non-null    object 
 10  premiere    513 non-null    object 
 11  imdb_score  513 non-null    float64
 12  language    513 non-null    object 
dtypes: float64(1), object(12)
memory usage: 56.1+ KB


In [13]:
# Contamos el número de filas duplicadas del dataframe
df.duplicated().sum()

0

In [14]:
# Contamos el número de nulos del dataframe
df.isnull().sum()

type            0
title           0
director       22
cast           91
country        14
date_added      0
rating          0
duration      237
listed_in       0
genre           0
premiere        0
imdb_score      0
language        0
dtype: int64

In [15]:
# Principales estadísticos de las columnas numéricas
df.describe().T

,count,mean,std,min,25%,50%,75%,max
imdb_score,513.0,6.210916,0.96885,2.5,5.6,6.3,6.9,9.0


In [16]:
# Principales estadísticos de las columnas categóricas
df.describe(include = "object").T

,count,unique,top,freq
type,513,1,Movie,513
title,513,513,Dick Johnson Is Dead,1
director,491,462,McG,3
cast,422,421,Shawn Mendes,2
country,499,81,United States,274
date_added,513,354,"October 18, 2019",5
rating,513,10,TV-MA,254
duration,276,86,98 min,20
listed_in,513,103,Documentaries,69
genre,513,106,Documentary,132


Vemos que para la columna de `type` solo tenemos un tipo, ¿merece la pena entonces mantener esta columna? La verdad es que no, así que vamos a eliminarla

In [17]:
df.drop(["type"], axis = 1, inplace = True)
df.head(2)

,title,director,cast,country,date_added,rating,duration,listed_in,genre,premiere,imdb_score,language
0,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",PG-13,90 min,Documentaries,Documentary,"October 2, 2020",7.5,English
1,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada","September 1, 2021",R,82 min,Action & Adventure,Action,"April 24, 2020",6.7,English


Dentro del análisis exploratorio es importante conocer todas las variables, con las categóricas es importante ver todas las categorías que tenemos para cada una de ellas y cuáles son sus frecuencias. 

In [18]:
# creemos un dataframe solo con las variables categóricas

df_cat = df.select_dtypes(include = "object")
df_cat.head()

,title,director,cast,country,date_added,rating,duration,listed_in,genre,premiere,language
0,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",PG-13,90 min,Documentaries,Documentary,"October 2, 2020",English
1,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada","September 1, 2021",R,82 min,Action & Adventure,Action,"April 24, 2020",English
2,Prime Time,Jakub Piątek,"Bartosz Bielenia, Magdalena Popławska, Andrzej...",NaN,"June 30, 2021",TV-MA,NaN,"Dramas, International Movies, Thrillers",Thriller,"April 14, 2021",Polish
3,Blue Miracle,Julio Quintana,"Jimmy Gonzales, Dennis Quaid, Anthony Gonzalez...",United States,"May 27, 2021",TV-PG,NaN,"Children & Family Movies, Dramas, Faith & Spir...",Drama,"May 27, 2021",English
4,Ghost Lab,Paween Purijitpanya,"Thanapob Leeratanakachorn, Paris Intarakomalya...",Thailand,"May 27, 2021",TV-MA,118 min,"Horror Movies, International Movies, Thrillers",Horror,"May 26, 2021",Thai


Si bien es cierto que hemos sacado todas la categóricas, ¿tiene sentido explorarlas todas? Podríamos pensar que variables como los títulos de cada peli no nos interesa, porque tienen demasiadas categorías. En este caso podremos eliminarlas de nuestro df_cat

In [19]:
df_cat.drop(["title"], axis = 1, inplace = True)
df_cat.head()

,director,cast,country,date_added,rating,duration,listed_in,genre,premiere,language
0,Kirsten Johnson,NaN,United States,"September 25, 2021",PG-13,90 min,Documentaries,Documentary,"October 2, 2020",English
1,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada","September 1, 2021",R,82 min,Action & Adventure,Action,"April 24, 2020",English
2,Jakub Piątek,"Bartosz Bielenia, Magdalena Popławska, Andrzej...",NaN,"June 30, 2021",TV-MA,NaN,"Dramas, International Movies, Thrillers",Thriller,"April 14, 2021",Polish
3,Julio Quintana,"Jimmy Gonzales, Dennis Quaid, Anthony Gonzalez...",United States,"May 27, 2021",TV-PG,NaN,"Children & Family Movies, Dramas, Faith & Spir...",Drama,"May 27, 2021",English
4,Paween Purijitpanya,"Thanapob Leeratanakachorn, Paris Intarakomalya...",Thailand,"May 27, 2021",TV-MA,118 min,"Horror Movies, International Movies, Thrillers",Horror,"May 26, 2021",Thai


In [20]:
for columna in df_cat.columns:
    print(f"la cantidad de valores únicos para la columna {columna.upper()} son {len(df_cat[columna].unique())} y estos valores son")
    (display(pd.DataFrame(df_cat[columna].value_counts())))
    print("---------------------------------------------")

la cantidad de valores únicos para la columna DIRECTOR son 463 y estos valores son


,director
McG,3
Amy Poehler,2
Christopher Guest,2
Noah Baumbach,2
Michael Tiddes,2
...,...
Gina Prince-Bythewood,1
Sue Ding,1
Stéphane de Freitas,1
Stefano Mordini,1


---------------------------------------------
la cantidad de valores únicos para la columna CAST son 422 y estos valores son


,cast
Shawn Mendes,2
"Bruce Willis, Kellan Lutz, Gina Carano, D.B. Sweeney, Joshua Mikel, Steve Coulter, Dan Bilzerian, Heather Johansen",1
Loudon Wainwright III,1
"Liam Neeson, James Franco, Tim Blake Nelson, Tom Waits, Zoe Kazan, Brendan Gleeson",1
"Madeline Brewer, Patch Darragh, Melora Walters, Devin Druid, Imani Hakim, Michael Dempsey, Flora Diaz, Samantha Robinson, Jessica Parker Kennedy, Quei Tann",1
...,...
"Marta Etura, Leonardo Sbaraglia, Carlos Librado ""Nene"", Francesc Orella, Imanol Arias, Álvaro Cervantes, Itziar Aizpuru, Benn Northover, Marta Larralde, Alicia Sánchez, Eduardo Rosa, Angel Alkain, Ana Wagener, Paco Tous, Patricia López Arnaiz, Pedro Casablanc",1
"Fulu Mugovhani, Tumi Morake, Bohang Moeko, Yonda Thomas",1
"Nawazuddin Siddiqui, Radhika Apte, Khalid Tyabji, Aditya Srivastava, Padmavati Rao, Shivani Raghuvanshi, Nishant Dahiya, Shweta Tripathi, Gyanendra Tripathi, Shreedhar Dubey, Swanand Kirkire, Riya Shukla, Tigmanshu Dhulia, Ila Arun, Natasha Rastogi",1
Nicolas Anelka,1


---------------------------------------------
la cantidad de valores únicos para la columna COUNTRY son 82 y estos valores son


...

---------------------------------------------
la cantidad de valores únicos para la columna PREMIERE son 350 y estos valores son


,premiere
"October 2, 2020",5
"November 1, 2019",5
"October 18, 2019",5
"April 10, 2020",4
"December 7, 2018",4
...,...
"January 17, 2020",1
"January 1, 2020",1
"December 26, 2019",1
"December 20, 2019",1


---------------------------------------------
la cantidad de valores únicos para la columna LANGUAGE son 37 y estos valores son


,language
English,352
Hindi,28
Spanish,26
French,18
Italian,14
Indonesian,9
Portuguese,9
English/Spanish,5
Japanese,5
Korean,5


---------------------------------------------


Al ver todos los valores únicos podemos ver que las columnas de `lister_in` y `genre` nos dan la misma información, así que procedemos a eliminar una de ellas: 

In [21]:
df.drop(["listed_in"], axis = 1, inplace = True)
df.head(2)

,title,director,cast,country,date_added,rating,duration,genre,premiere,imdb_score,language
0,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",PG-13,90 min,Documentary,"October 2, 2020",7.5,English
1,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada","September 1, 2021",R,82 min,Action,"April 24, 2020",6.7,English


## Limpieza `director`, `cast`, `country`

Estas columnas eran las que tenían valores nulos

In [22]:
df.isnull().sum()[df.isnull().sum() > 0]

director     22
cast         91
country      14
duration    237
dtype: int64

In [23]:
# eliminamos los valores nulos de las columnas "director", "cast", "country" por Unknown
df[["director", "cast", "country"]]= df[["director", "cast", "country"]].fillna("Unknown")

In [24]:
df.head(2)

,title,director,cast,country,date_added,rating,duration,genre,premiere,imdb_score,language
0,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",PG-13,90 min,Documentary,"October 2, 2020",7.5,English
1,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada","September 1, 2021",R,82 min,Action,"April 24, 2020",6.7,English


Imagineos que queremos cambiar el orden de las columnas, cambiemos el orden usando el método `reindex`. 

In [25]:
# creamos una lista con el nuevo orden que queremos
new_order = ['title', 'director', 'cast', 'country', 'language', 'rating', 'genre',
       'premiere', 'date_added', 'duration', 'imdb_score' ]

# aplicamos el método reindex
df = df.reindex(columns=new_order)
df.head(2)

,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score
0,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,English,PG-13,Documentary,"October 2, 2020","September 25, 2021",90 min,7.5
1,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada",English,R,Action,"April 24, 2020","September 1, 2021",82 min,6.7


# Limpiamos la columna de `duration` 

In [26]:
# lo primero que hacemos es ver los valores únicos

df["duration"].unique()

array(['90 min', '82 min', nan, '118 min', '73 min', '148 min', '140 min',
       '107 min', '98 min', '128 min', '121 min', '142 min', '143 min',
       '84 min', '132 min', '55 min', '112 min', '113 min', '81 min',
       '120 min', '119 min', '138 min', '127 min', '116 min', '32 min',
       '54 min', '8 min', '71 min', '9 min', '48 min', '117 min',
       '83 min', '42 min', '13 min', '124 min', '150 min', '50 min',
       '123 min', '130 min', '110 min', '47 min', '126 min', '104 min',
       '122 min', '28 min', '20 min', '80 min', '135 min', '17 min',
       '41 min', '133 min', '156 min', '149 min', '5 min', '75 min',
       '16 min', '111 min', '85 min', '145 min', '79 min', '25 min',
       '129 min', '137 min', '209 min', '40 min', '141 min', '52 min',
       '22 min', '38 min', '64 min', '10 min', '46 min', '59 min',
       '30 min', '49 min', '60 min', '26 min', '63 min', '74 min',
       '153 min', '35 min', '144 min', '77 min', '12 min', '78 min',
       '53 min', '131 m

In [27]:
# como todos los valores tienen la misma estructura, lo único que tenemos que hacer es quitarle el min. 

df["duration"] = df["duration"].str.split(" ", expand = True).get(0)
df.head()

,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score
0,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,English,PG-13,Documentary,"October 2, 2020","September 25, 2021",90,7.5
1,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada",English,R,Action,"April 24, 2020","September 1, 2021",82,6.7
2,Prime Time,Jakub Piątek,"Bartosz Bielenia, Magdalena Popławska, Andrzej...",Unknown,Polish,TV-MA,Thriller,"April 14, 2021","June 30, 2021",NaN,5.7
3,Blue Miracle,Julio Quintana,"Jimmy Gonzales, Dennis Quaid, Anthony Gonzalez...",United States,English,TV-PG,Drama,"May 27, 2021","May 27, 2021",NaN,6.7
4,Ghost Lab,Paween Purijitpanya,"Thanapob Leeratanakachorn, Paris Intarakomalya...",Thailand,Thai,TV-MA,Horror,"May 26, 2021","May 27, 2021",118,5.2


In [28]:
# esta columna era de tipo object, veamos ahora si es de tipo numérico
df.dtypes

title          object
director       object
cast           object
country        object
language       object
rating         object
genre          object
premiere       object
date_added     object
duration       object
imdb_score    float64
dtype: object

In [29]:
# la columna "duration" sigue siendo de tipo object, pero esto no tiene sentido, convirtamosla a numérica
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')

In [30]:
# confirmamos la conversion de los valores
df.dtypes

title          object
director       object
cast           object
country        object
language       object
rating         object
genre          object
premiere       object
date_added     object
duration      float64
imdb_score    float64
dtype: object

In [31]:
# para los nulos de duración, los reemplazaremos por la mediana
df[df["duration"].isnull()].head()



,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score
2,Prime Time,Jakub Piątek,"Bartosz Bielenia, Magdalena Popławska, Andrzej...",Unknown,Polish,TV-MA,Thriller,"April 14, 2021","June 30, 2021",NaN,5.7
3,Blue Miracle,Julio Quintana,"Jimmy Gonzales, Dennis Quaid, Anthony Gonzalez...",United States,English,TV-PG,Drama,"May 27, 2021","May 27, 2021",NaN,6.7
5,Baggio: The Divine Ponytail,Letizia Lamartire,"Andrea Arcangeli, Valentina Bellè, Andrea Penn...",Italy,Italian,TV-MA,Biopic,"May 26, 2021","May 26, 2021",NaN,6.2
11,The Woman in the Window,Joe Wright,"Amy Adams, Gary Oldman, Anthony Mackie, Fred H...",United States,English,R,Psychological thriller,"May 14, 2021","May 14, 2021",NaN,5.7
12,Oxygen,Alexandre Aja,"Mélanie Laurent, Mathieu Amalric, Malik Zidi","France, United States",French,TV-14,Science fiction thriller,"May 12, 2021","May 12, 2021",NaN,6.5


In [32]:
# en este caso sólo tenemos valores nulos en "Movies", por lo que necesitamos calcular la duración mediana para este tipo de información
median_movies = df["duration"].median()

# Sustituir los valores nulos de la columna "duration" por el valor medio de la columna "duración" para el tipo "Movie".
df["duration"] = df["duration"].fillna(median_movies)



In [33]:
# check the null values again
df.isnull().sum()

title         0
director      0
cast          0
country       0
language      0
rating        0
genre         0
premiere      0
date_added    0
duration      0
imdb_score    0
dtype: int64

# Limpiamos `date_added` y `premiere`

In [34]:
df.head(2)

,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score
0,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,English,PG-13,Documentary,"October 2, 2020","September 25, 2021",90.0,7.5
1,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada",English,R,Action,"April 24, 2020","September 1, 2021",82.0,6.7


In [35]:
# convertimos al fecha a datetime
df["premiere"] = pd.to_datetime(df[ "premiere"])
df["date_added"] = pd.to_datetime(df["date_added"])

In [36]:
# confirmamos la transformación
df.dtypes

title                 object
director              object
cast                  object
country               object
language              object
rating                object
genre                 object
premiere      datetime64[ns]
date_added    datetime64[ns]
duration             float64
imdb_score           float64
dtype: object

In [37]:
df.head(2)

,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score
0,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,English,PG-13,Documentary,2020-10-02,2021-09-25,90.0,7.5
1,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7


## Clean `cast` y `director`


In [38]:
df_act_dire = df.copy()

In [39]:
df["cast"].unique().tolist()[1]

'Bruce Willis, Kellan Lutz, Gina Carano, D.B. Sweeney, Joshua Mikel, Steve Coulter, Dan Bilzerian, Heather Johansen'

In [40]:
# lo primero que tenemos que hacer es separar cada ector y actriz

df_act_dire["cast"] = df_act_dire["cast"].str.split(",")
df_act_dire.head(2)

,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score
0,Dick Johnson Is Dead,Kirsten Johnson,[Unknown],United States,English,PG-13,Documentary,2020-10-02,2021-09-25,90.0,7.5
1,Extraction,Steven C. Miller,"[Bruce Willis, Kellan Lutz, Gina Carano, D....","United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7


In [41]:
# separamos cada actor en una fila nueva
df_actores = df_act_dire.explode("cast")
df_actores.head()

,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score
0,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,English,PG-13,Documentary,2020-10-02,2021-09-25,90.0,7.5
1,Extraction,Steven C. Miller,Bruce Willis,"United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7
1,Extraction,Steven C. Miller,Kellan Lutz,"United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7
1,Extraction,Steven C. Miller,Gina Carano,"United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7
1,Extraction,Steven C. Miller,D.B. Sweeney,"United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7


Aplicamos la misma lógica para la columna de directores

In [42]:
df["director"].unique().tolist()[16]

'Robert Pulcini, Shari Springer Berman'

In [43]:
df_act_dire["director"] = df_act_dire["director"].str.split(",")
df_act_dire.head(2)

,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score
0,Dick Johnson Is Dead,[Kirsten Johnson],[Unknown],United States,English,PG-13,Documentary,2020-10-02,2021-09-25,90.0,7.5
1,Extraction,[Steven C. Miller],"[Bruce Willis, Kellan Lutz, Gina Carano, D....","United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7


In [44]:
# separamos cada actor en una fila nueva
df_directores = df_act_dire.explode("director")
df_directores.head(3)

,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score
0,Dick Johnson Is Dead,Kirsten Johnson,[Unknown],United States,English,PG-13,Documentary,2020-10-02,2021-09-25,90.0,7.5
1,Extraction,Steven C. Miller,"[Bruce Willis, Kellan Lutz, Gina Carano, D....","United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7
2,Prime Time,Jakub Piątek,"[Bartosz Bielenia, Magdalena Popławska, Andr...",Unknown,Polish,TV-MA,Thriller,2021-04-14,2021-06-30,101.0,5.7


## Limpiamos `genre` 


In [45]:
# vemos los valores únicos

print(df["genre"].unique().tolist())

['Documentary', 'Action', 'Thriller', 'Drama', 'Horror', 'Biopic', 'Zombie/Heist', 'Comedy', 'Crime drama', 'Psychological thriller', 'Science fiction thriller', 'Animated musical comedy', 'Psychological thriller drama', 'Superhero-Comedy', 'Romantic comedy', 'Christian musical', 'Hidden-camera prank comedy', 'Comedy-drama', 'Romantic teen drama', 'Romantic drama', 'Science fiction', 'Action/Science fiction', 'Animation / Short', 'Superhero', 'Aftershow / Interview', 'Musical', 'Animation', 'Concert Film', 'Christmas comedy', 'Stop Motion', 'Family/Christmas musical', 'Anthology/Dark comedy', 'Mystery', 'Romantic comedy/Holiday', 'Variety show', 'Animation/Musical/Adventure', 'Romantic thriller', 'Comedy/Fantasy/Family', 'Horror comedy', 'Action comedy', 'Family', 'Comedy/Horror', 'Drama/Horror', 'Animation/Superhero', 'Dance comedy', 'Superhero/Action', 'Romantic teenage drama', 'Musical comedy', 'Family/Comedy-drama', 'Romance', 'Anime/Fantasy', 'War drama', 'Heist film/Thriller', 'A

In [46]:
# para facilitar la limpieza lo primero qye hacemos es poner todos los géneros en minúscula

df["genre"] = df["genre"].str.lower()

In [47]:
# comprobamos que están en minúsculas

print(df["genre"].unique().tolist())

['documentary', 'action', 'thriller', 'drama', 'horror', 'biopic', 'zombie/heist', 'comedy', 'crime drama', 'psychological thriller', 'science fiction thriller', 'animated musical comedy', 'psychological thriller drama', 'superhero-comedy', 'romantic comedy', 'christian musical', 'hidden-camera prank comedy', 'comedy-drama', 'romantic teen drama', 'romantic drama', 'science fiction', 'action/science fiction', 'animation / short', 'superhero', 'aftershow / interview', 'musical', 'animation', 'concert film', 'christmas comedy', 'stop motion', 'family/christmas musical', 'anthology/dark comedy', 'mystery', 'romantic comedy/holiday', 'variety show', 'animation/musical/adventure', 'romantic thriller', 'comedy/fantasy/family', 'horror comedy', 'action comedy', 'family', 'comedy/horror', 'drama/horror', 'animation/superhero', 'dance comedy', 'superhero/action', 'romantic teenage drama', 'musical comedy', 'family/comedy-drama', 'romance', 'anime/fantasy', 'war drama', 'heist film/thriller', 'a

In [48]:
df['genre2'] = df['genre'].str.split("/")
df.head(2)


,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score,genre2
0,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,English,PG-13,documentary,2020-10-02,2021-09-25,90.0,7.5,[documentary]
1,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada",English,R,action,2020-04-24,2021-09-01,82.0,6.7,[action]


In [49]:
# usando la librería Counter, calculamos los géneros más comunes.
cuenta_generos = Counter(genres for genres in df['genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

Hay 106 generos diferentes.


In [50]:
comunes = dict(cuenta_generos.most_common(10))
comunes

{'documentary': 132,
 'drama': 73,
 'comedy': 42,
 'romantic comedy': 35,
 'thriller': 33,
 'comedy-drama': 14,
 'crime drama': 10,
 'horror': 9,
 'biopic': 8,
 'action': 7}

In [51]:
fuzz.ratio('romantic comedy', "documentary")

38

In [52]:
def generos_(col, generos_comunes):
    maximo = 0
    for key in generos_comunes.keys():
        parecido = fuzz.ratio(col, key)
        if parecido > maximo:
            maximo = parecido
            genero = key
            
    # nos aseguramos de que los géneros se parezcan en al menos un 90%        
    if maximo > 50:
        return genero
    else:
        return "Other"
            

In [53]:
df["genre2"] = df.apply(lambda x: generos_(x["genre"], comunes), axis = 1)

In [54]:
df["genre2"].value_counts()

documentary        133
drama               78
Other               68
romantic comedy     62
comedy              52
thriller            51
comedy-drama        19
crime drama         17
action              14
horror              11
biopic               8
Name: genre2, dtype: int64

# Top 10 directores y actores

In [55]:
df_actores.head()

,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score
0,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,English,PG-13,Documentary,2020-10-02,2021-09-25,90.0,7.5
1,Extraction,Steven C. Miller,Bruce Willis,"United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7
1,Extraction,Steven C. Miller,Kellan Lutz,"United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7
1,Extraction,Steven C. Miller,Gina Carano,"United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7
1,Extraction,Steven C. Miller,D.B. Sweeney,"United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7


In [56]:
top_actores = df_actores["cast"].value_counts().reset_index()
top_actores.head()

,index,cast
0,Unknown,91
1,Adam Sandler,7
2,Maya Rudolph,6
3,Andrew Bachelor,5
4,Lakeith Stanfield,4


In [57]:
lista_top_actores = top_actores.iloc[1:11,:]["index"].tolist()
lista_top_actores = [actor.strip() for actor in lista_top_actores]
print(lista_top_actores)

['Adam Sandler', 'Maya Rudolph', 'Andrew Bachelor', 'Lakeith Stanfield', 'Jacki Weaver', 'Ken Marino', 'Robbie Amell', 'Zachary Quinto', 'Nick Swardson', 'Rob Schneider']


In [58]:
df_directores.head(2)

,title,director,cast,country,language,rating,genre,premiere,date_added,duration,imdb_score
0,Dick Johnson Is Dead,Kirsten Johnson,[Unknown],United States,English,PG-13,Documentary,2020-10-02,2021-09-25,90.0,7.5
1,Extraction,Steven C. Miller,"[Bruce Willis, Kellan Lutz, Gina Carano, D....","United States, United Kingdom, Canada",English,R,Action,2020-04-24,2021-09-01,82.0,6.7


In [59]:
top_directores = df_directores["director"].value_counts().reset_index()
top_directores.head()

,index,director
0,Unknown,22
1,McG,3
2,Blair Simmons,2
3,John Schultz,2
4,Noah Baumbach,2


In [60]:
lista_top_directores = top_directores.iloc[1:11,:]["index"].tolist()
lista_top_directores = [director.strip() for director in lista_top_directores]
print(lista_top_directores)

['McG', 'Blair Simmons', 'John Schultz', 'Noah Baumbach', 'Karan Johar', 'Dibakar Banerjee', 'Julien Leclercq', 'Amy Poehler', 'Kyle Newacheck', 'Peter Sullivan']


In [61]:
# guardamos estos resultados para usarlos mañana
with open ('../data/pickle/top10_directors.pickle', "wb") as dire:
    pickle.dump(lista_top_directores, dire)
    
with open ('../data/pickle/top10_actors.pickle', "wb") as acto:
    pickle.dump(lista_top_actores, acto)